In [1]:
%pylab inline
from numpy import load
from KDTreeEncoding import *
from glob import glob

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
current=%pwd
current

'/Users/yoavfreund/projects/Poverty_Analysis/KD-trees'

In [3]:
_dir='/Users/yoavfreund/datasets/poverty_v1.1/images/'
files=glob(f'{_dir}*.npz')
len(files)

19669

In [4]:
%%time
train_size,tree=train_encoder(files[:100])

CPU times: user 1.58 s, sys: 181 ms, total: 1.76 s
Wall time: 1.84 s


In [5]:
train_size

5017600

In [6]:
poverty_dir='/Users/yoavfreund/datasets/poverty_v1.1'
!ls $poverty_dir

README.md                africa_poverty_git       anon_images.tgz          images
RELEASE_v1.1.txt         anon_images              dhs_incountry_folds.pkl  poverty_deepLearning.pdf


In [7]:
import pandas as pd

In [8]:
T=pd.read_csv(f'{current}/../tables/dhs_metadata.csv')

rural_tanzania=pd.DataFrame(T[(T['country']=='tanzania') & (T['urban']==False)])

rural_tanzania.head(2)

,lat,lon,wealthpooled,country,year,urban,nl_mean,nl_center,households
15410,-1.369792,34.660076,-0.761560,tanzania,2010,False,-0.173862,-0.173862,22
15411,-1.400853,34.556240,-0.687581,tanzania,2010,False,-0.018425,0.085971,17


In [9]:
rural_tanzania['rich']=rural_tanzania['wealthpooled']>-0.5
rural_tanzania['index']=rural_tanzania.index
rural_tanzania.head()

,lat,lon,wealthpooled,country,year,urban,nl_mean,nl_center,households,rich,index
15410,-1.369792,34.660076,-0.761560,tanzania,2010,False,-0.173862,-0.173862,22,False,15410
15411,-1.400853,34.556240,-0.687581,tanzania,2010,False,-0.018425,0.085971,17,False,15411
15412,-3.734950,35.566425,-0.750120,tanzania,2010,False,-0.173862,-0.173862,20,False,15412
15413,-5.035908,38.362194,-0.737002,tanzania,2010,False,-0.173862,-0.173862,21,False,15413
15414,-5.440775,38.285057,-0.518625,tanzania,2010,False,-0.173862,-0.173862,19,False,15414


In [10]:
rural_tanzania.shape

(1216, 11)

In [11]:
import pickle as pkl


In [12]:
# encode a binary array as an int.
def bin2int(c):
    ans=0
    for j in range(len(c)):
        bit=int(c[-(j+1)])
        #print(j,bit)
        ans+=bit*(2**j)
    return ans

file_template='/Users/yoavfreund/datasets/poverty_v1.1/partitioned_images/rural/tanzania/%d/landsat_poverty_img_%d.npz'
Encoding={}
j=1
Mat=np.zeros([rural_tanzania.shape[0],2**9],dtype=np.float32)
for i,row in rural_tanzania.iterrows():
    year=row['year']
    filename=file_template%(year,i)
    if not filename in files:
        break
    code = encode_image(filename,tree)
    V=np.zeros(2**9)
    for c,a in code:
        V[bin2int(c)]=a
    Encoding[i]=V
    if(j %10==0):
        print(j,i,end='\r')
    j+=1

len(Encoding)

0

In [13]:
%pylab inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from lib import XGBHelper as xgbh

from lib.logger import logger

%load_ext autoreload

%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Users/yoavfreund/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [14]:
just_label=rural_tanzania[['rich']]
rows=just_label.shape[0]
just_label.head()

,rich
15410,False
15411,False
15412,False
15413,False
15414,False


In [15]:
data=np.zeros([rows,2**9+1]) #code length +1 for label

count=0
for i,row in just_label.iterrows():
    label=row['rich']*1
    data[count,-1]=label
    data[count,:-1]=Encoding[i]
    count+=1

KeyError: 15410

In [16]:
import pickle as pkl
pkl.dump(data,open('RuralTanzania.pkl','wb'))

In [17]:
!ls -lh RuralTanzania.pkl

-rw-r--r--  1 yoavfreund  wheel   4.8M May 12 14:42 RuralTanzania.pkl


In [18]:
!rm tanzania.pkl

rm: tanzania.pkl: No such file or directory


In [17]:
%pwd

'/Users/yoavfreund/projects/Poverty_Analysis/KD-trees'